In [5]:
##AWS 람다 수동 테스트용 코드


import requests

url = "https://buqpkgyu4lx35cye3vku6i7xwy0vqcol.lambda-url.ap-northeast-2.on.aws/"
r = requests.post(url, json={"issue_key": "PAYDAY-5", "update": "false"})
print(r.status_code, r.text)

200 {"issue_key": "PAYDAY-5", "results": {"summary": {"original": "[#41.2][Heist] 의료병 12레벨 화염병 범위 증가 스킬의 효과가 없는 현상 / The effect of Support level 12 Molotov range increase skill is not applied", "translated": ""}, "description": {"original": "*[QA 환경 / QA Environment]*\n\npc-heistroyalenew\n*[상세 설명 / Details]*\n\n의료병 12레벨 패시브 스킬인 ‘투척무기의 효과 범위가 증가합니다’ 스킬이 동작하지 않는 현상입니다.\n\n1레벨 의료병과 13레벨 의료병이 동일한 위치에 화염병을 수 회 이상 투척하며 비교하였는데, 번진 화염의 범위에 유의미한 차이를 발견하지 못하였습니다.\n\n* 좌 : 1레벨 의료병 / 우 : 13레벨 의료병\n\n{color:#4c9aff}This is an issue where the level 12 Support passive skill, “Increases the effect radius of thrown weapons,” does not work.{color}\n{color:#4c9aff}We compared by throwing Molotovs multiple times at the same position with a level 1 Support and a level 13 Support, but could not find any meaningful difference in the spread fire range.{color}\n\n* {color:#4c9aff}Left: level 1 Support / Right: level 13 Support{color}\n!image-20251226-053016.png|width=83%,alt=\"image-20251226-053016.png\"!\n\n